In [3]:
! mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/Colab\ Notebooks/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download kaustubhb999/tomatoleaf

 98% 175M/179M [00:01<00:00, 173MB/s]
100% 179M/179M [00:01<00:00, 152MB/s]


In [ ]:
! unzip /content/tomatoleaf.zip

In [8]:
import os
num_classes=len(os.listdir("/content/tomato/train"))
num_classes

10

In [9]:
from keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
training_set = train_datagen.flow_from_directory(
    '/content/tomato/train',
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical' )

test_set=test_datagen.flow_from_directory(
    '/content/tomato/val',
    target_size=(128,128),
    batch_size=64,
    class_mode='categorical' )

Found 10000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [13]:
label_map = (training_set.class_indices)

print(label_map)

{'Tomato___Bacterial_spot': 0, 'Tomato___Early_blight': 1, 'Tomato___Late_blight': 2, 'Tomato___Leaf_Mold': 3, 'Tomato___Septoria_leaf_spot': 4, 'Tomato___Spider_mites Two-spotted_spider_mite': 5, 'Tomato___Target_Spot': 6, 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato___Tomato_mosaic_virus': 8, 'Tomato___healthy': 9}


In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from keras.losses import categorical_crossentropy
import matplotlib.pyplot as plt
import numpy as np
import os

In [23]:
def LeNet():
    model = models.Sequential()

    model.add(layers.Conv2D(6, (5, 5), activation='relu', input_shape=(128,128,3)))
    model.add(layers.AveragePooling2D())

    model.add(layers.Conv2D(16, (5, 5), activation='relu'))
    model.add(layers.AveragePooling2D())

    model.add(layers.Flatten())

    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dense(84, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    return model

In [24]:
model = LeNet()

model.compile(optimizer=Adam(learning_rate=0.001), loss=categorical_crossentropy, metrics=['accuracy'])

In [25]:
model.fit_generator(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=10,
    validation_data=test_set,
    validation_steps=len(test_set)
)

<ipython-input-25-8fe9bdae9929>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
157/157 [==============================] - 60s 343ms/step - loss: 1.6667 - accuracy: 0.4113 - val_loss: 1.2107 - val_accuracy: 0.5770
Epoch 2/10
157/157 [==============================] - 51s 328ms/step - loss: 0.9878 - accuracy: 0.6597 - val_loss: 0.9608 - val_accuracy: 0.6430
Epoch 3/10
157/157 [==============================] - 54s 346ms/step - loss: 0.7607 - accuracy: 0.7333 - val_loss: 0.7999 - val_accuracy: 0.7250
Epoch 4/10
157/157 [==============================] - 50s 320ms/step - loss: 0.6454 - accuracy: 0.7740 - val_loss: 0.6412 - val_accuracy: 0.7920
Epoch 5/10
157/157 [==============================] - 53s 338ms/step - loss: 0.5170 - accuracy: 0.8220 - val_loss: 0.5632 - val_accuracy: 0.8140
Epoch 6/10
157/157 [==============================] - 51s 325ms/step - loss: 0.4940 - accuracy: 0.8320 - val_loss: 0.5269 - val_accuracy: 0.8110
Epoch 7/10
157/157 [==============================] - 51s 322ms/step - loss: 0.4425 - accuracy: 0.8449 - val_loss: 0.7276 - val_ac

In [26]:
model.save('tomato_disease_classifier_LeNet.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
model = load_model('tomato_disease_classifier_LeNet.h5')

predictions = model.predict(test_set, steps=len(test_set))
predictions = np.argmax(predictions, axis=1)

y_true = test_set.classes
y_pred = predictions

accuracy = accuracy_score(y_true, y_pred)

print('Accuracy:', accuracy)

16/16 [==============================] - 1s 69ms/step
Accuracy: 0.115
